In [1]:
# FILE NAME: Collecting data.ipynb
# PROGRAMMER: VG6
# DATE: 24.03.2020
# Purpose: To collect valuable information from open edu recourse


In [2]:
# importing modules
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import re

In [3]:

# initializing path to firefox driver
firefox_driver = 'C:\\Users\Gololobov\Documents\GitHub\Collecting_data_openedu\geckodriver.exe'

# initializing firefox options instance 
options = webdriver.FirefoxOptions()

# initializing headless options
options.add_argument('headless')

# initializing browser with driver and options
browser = webdriver.Firefox(executable_path=firefox_driver, options=options)


In [4]:
# initializing html tags patterns
tags = re.compile(r'<[^>]+>')

# Removing html tags from text function
# Arguments: 
#   - text we want to remove tags from
#      text : str
# Returns:
#   - string with removed html texts
#      (str)
def remove_html_tags(text : str) -> str:
    return tags.sub('', text)

In [5]:
# Getting particular type of courses for fixed number of pages function
# Arguments: 
#   - None
# Returns:
#   - Data Frame with info about this group of courses
#       (pd.DataFrame)
def get_courses() -> pd.DataFrame:
    # initializing link of courses page
    courses_link = 'https://openedu.ru/course/'
    
    # creating list of Data Frames representing each page
    each_page_courses_dfs = []
    
    # getting courses page 
    browser.get(courses_link)

    # initializing beautiful soup class instance to parse current page
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    # getting number of courses
    number_of_courses = soup.find('h1').text.split()[1]
    
    # initizliing starting page
    page = 0
    
    # initializing courses counter
    current_courses_count = 0
    
    print(number_of_courses)
    
    # collecting info about courses
    while current_courses_count < int(number_of_courses):
        # initializing full page with link
        courses_link_with_page = courses_link + '#page=' + str(page)
        
        # getting training group page
        browser.get(courses_link_with_page)

        # initializing beautiful soup class instance to parse current page
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        
        # getting current training group 
        training_group = soup.find_all('a', {'class' : 'chosen-single'})[1].text
        
        # getting cources blocks
        courses_blokcs = soup.find_all('div', {'class': 'col-md-4 col-sm-6 col-xs-12 col'})
        
        # declarating links to courses list
        links_to_courses = []

        # declarating names of the courses list
        names_of_courses = []

        # getting link of courses
        for course in courses_blokcs:
            # getting course title div
            course_title = course.find('div', {'class' : 'course-title'})

            # adding reference to the course 
            links_to_courses.append('https://openedu.ru' + course_title.find('a')['href'])

            # adding name of the course
            names_of_courses.append(course_title.text)
            
            
            # declarating descriptions dfs of courses
            descriptions_dfs = []

            for (course_link, course_name) in zip(links_to_courses, names_of_courses):
                descriptions_dfs.append(get_course_info(course_link, course_name))
                
            # initializing Data Frame with information of all
            # courses on the current page
            page_courses_info = pd.DataFrame()

            # concating all DataFrames we get on previous step
            for data_frame in descriptions_dfs:
                # concatinating all courses info dataframes
                page_courses_info = pd.concat([page_courses_info, data_frame], axis=0, ignore_index=True)
        
        # adding amount of courses on the page
        current_courses_count += len(courses_blokcs)
        print(current_courses_count)

        # adding page dataframe to list
        each_page_courses_dfs.append(page_courses_info)
        
        page += 1
        
    # initializing DataFrame with information od all
    # courses on all pages
    all_courses_info = pd.DataFrame()
    
    # concating all DataFrames we get on previous step
    for data_frame in each_page_courses_dfs:
        # concatinating all courses info dataframes
        all_courses_info = pd.concat([all_courses_info, data_frame], axis=0, ignore_index=True)
    
    return all_courses_info

In [6]:
# Getting course information function
# Arguments:
#   - Link to course and it's name
#       link_to_the_course : str, course_name : str
# Returns:
#   - DataFrame with full information about the course
#       (pd.DataFrame)
def get_course_info(link_to_the_course : str, course_name : str) -> pd.DataFrame:
    # getting course page
    browser.get(link_to_the_course)

    # initializing beautiful soup class instance to parse current page
    soup = BeautifulSoup(browser.page_source, 'html.parser')

    # getting description content
    course_description = soup.find('div', {'class' : 'col-sm-8 issue'})

    # getting header of course
    course_header = soup.find_all('h2', {'id': re.compile('.')})

    # going through all header and replacing them with
    # special symbol #
    for header in course_header:
        pattern = re.compile(str(header))
        course_description = re.sub(pattern, '!!!', str(course_description))

    # splitting course description to retrieve 
    # info under headers
    splitted_description = course_description.split('!!!')

    # initializing course_headers_names
    course_headers_names = []

    # getting names of headers
    for header in course_header:
        # getting text header and making it lower case
        header_text_lower = header.text.lower()

        # adding headers text to list
        course_headers_names.append(header_text_lower)

    # initializing columns of data frame
    df_columns = ['название'] + ['ссылка'] + course_headers_names
    
    cleaned_description = [remove_html_tags(x) for x in splitted_description]
    #cleaned_description = [re.sub(r'\n', '', x) for x in cleaned_description]
    
    # initilizing data of data frame
    df_data = [course_name] + [link_to_the_course] + cleaned_description[1:]
    
    # initliazing DataFrame for current course
    course_data = pd.DataFrame([df_data], columns=df_columns)
    
    return course_data

In [7]:
# Generating Data Frame only with valuable information function
# Arguments:
#  - Data Frame we want to change
#      data : pd.DataFrame
# Returns:
#  - Data Fram with only valuable information for us
#      (pd.DataFrame)
def generate_final_df(data : pd.DataFrame) -> pd.DataFrame:
    # initializing resulted data frame
    resulted_df = pd.DataFrame()
    
    # adding courses links
    resulted_df['URL'] = data['ссылка']
    
    # adding courses names
    resulted_df['Название курса'] = data['название']
    
    # adding requerments
    resulted_df['Входные требования'] = data['формируемые компетенции'] + data['требования']+ data['знания'] + data['умения'] + data['навыки']
    
    # adding content 
    resulted_df['Содержание курса'] = data['программа курса']
    
    # adding training areas
    resulted_df['Направления подготовки'] = data['направления подготовки']
    
    return resulted_df

In [8]:
courses_df = get_courses()
courses_df = courses_df.fillna('')


503


KeyboardInterrupt: 

In [21]:
courses_df = courses_df[:503]
courses_df.to_csv(r'Z:\data_openedu\courses_data.csv')
courses_df

,название,ссылка,о курсе,формат,требования,направления подготовки,информационные ресурсы,программа курса,результаты обучения,формируемые компетенции,знания,умения,навыки
0,Персидский язык. Разговорный курс. Часть 2.,https://openedu.ru/course/spbu/PERS2/,"\nКурс может быть полезен как специалистам, чь...",\nДистанционный\n,\nВладение базовой лексикой персидского языка ...,\n45.00.00 Языкознание и литературоведение\n\n\n,,,,,,,
1,Радиационная экология,https://openedu.ru/course/mephi/mephi_res/,\nКурс «Радиационная экология» рекомендуется к...,"\nВ состав курса входят 5 тем, каждой из котор...","\nСпециальных умений и навыков, без которых не...",\n14.03.02 Ядерные физика и технологии\n\n14.0...,\nСельскохозяйственная радиоэкология / Под ред...,\nТема 1. Радиационная экология: история разви...,\nПонимание закономерностей поведения радионук...,\n\n\n\n\nРО-1\n\n\nПонимание закономерностей ...,\nЗнание принципов проведения мероприятий по с...,\nСпособность оценивать риск и определять меры...,\nСпособностью к анализу технических и расчетн...
2,Самолёт: от пассажира к инженеру,https://openedu.ru/course/ssau/SPI/,\nВ задачи данного курса входит: \n &gt...,\nЕженедельные занятия включают в себя просмот...,\nДля прохождения курса требуется знание основ...,\n24.00.00 Авиационная и ракетно-космическая т...,,\nМодуль 1. Что такое качество и как им управл...,\nВ результате освоения курса «Самолёт: от пас...,\n\nСпособность применять знание подходов к уп...,,,
3,Эмоциональный интеллект,https://openedu.ru/course/misis/EMQ/,\nОсновными задачами данного курса являются:\n...,\nВ состав курса входят видео-лекции продолжит...,"\nКурс рассчитан на широкий круг участников, с...",\n38.03.01 Экономика\n\n38.03.02 Менеджмент\n\...,"\n\n\nГолви, Т. Стресс как внутренняя игра: ка...",\nСтруктура курса: Курс состоит из 10 недель и...,\nВ результате освоения курса у обучающихся фо...,\nКурс направлен на формирование универсальных...,,,
4,Теория вероятностей,https://openedu.ru/course/eltech/probability_t...,\nЦель онлайн-курса – изучение вероятностных з...,\nКурс включает:\n\nтематические видеолекции;\...,\nКурс рассчитан на студентов бакалавриата и с...,\n01.03.02 Прикладная математика и информатика...,\n\nБородин А.Н. Элементарный курс теории веро...,\nВведение\nТема 1. Случайные события\nОсновны...,"\nВ результате освоения курса, обучающийся спо...",,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,Эмоциональный интеллект,https://openedu.ru/course/misis/EMQ/,\nОсновными задачами данного курса являются:\n...,\nВ состав курса входят видео-лекции продолжит...,"\nКурс рассчитан на широкий круг участников, с...",\n38.03.01 Экономика\n\n38.03.02 Менеджмент\n\...,"\n\n\nГолви, Т. Стресс как внутренняя игра: ка...",\nСтруктура курса: Курс состоит из 10 недель и...,\nВ результате освоения курса у обучающихся фо...,\nКурс направлен на формирование универсальных...,,,
499,Электричество и магнетизм,https://openedu.ru/course/mipt/ELEC/,\nКурс «Электричество и магнетизм» рассчитан н...,"\nКурс рассчитан на 14 недель, из которых 12 у...",\nСлушателям курса необходимо владеть знаниями...,\n03.00.00 Физика и астрономия\n\n04.00.00 Хим...,\nОсновная литература:\n\nСивухин Д.В. Общий к...,\n\nЭлектрические заряды и электрическое поле....,\nБазовые знания:\n\nфизические явления и зако...,,,,
500,Радиационная экология,https://openedu.ru/course/mephi/mephi_res/,\nКурс «Радиационная экология» рекомендуется к...,"\nВ состав курса входят 5 тем, каждой из котор...","\nСпециальных умений и навыков, без которых не...",\n14.03.02 Ядерные физика и технологии\n\n14.0...,\nСельскохозяйственная радиоэкология / Под ред...,\nТема 1. Радиационная экология: история разви...,\nПонимание закономерностей поведения радионук...,\n\n\n\n\nРО-1\n\n\nПонимание закономерностей ...,\nЗнание принципов проведения мероприятий по с...,\nСпособность оценивать риск и определять меры...,\nСпособностью к анализу технических и расчетн...
501,Экономические основы транспортной деятельности,https://

In [22]:
final_df = generate_final_df(courses_df)

In [24]:
final_df.to_csv(r'Z:\data_openedu\final_data.csv')

In [25]:
final_df

,URL,Название курса,Входные требования,Содержание курса,Направления подготовки
0,https://openedu.ru/course/spbu/PERS2/,Персидский язык. Разговорный курс. Часть 2.,\nВладение базовой лексикой персидского языка ...,,\n45.00.00 Языкознание и литературоведение\n\n\n
1,https://openedu.ru/course/mephi/mephi_res/,Радиационная экология,\n\n\n\n\nРО-1\n\n\nПонимание закономерностей ...,\nТема 1. Радиационная экология: история разви...,\n14.03.02 Ядерные физика и технологии\n\n14.0...
2,https://openedu.ru/course/ssau/SPI/,Самолёт: от пассажира к инженеру,\n\nСпособность применять знание подходов к уп...,\nМодуль 1. Что такое качество и как им управл...,\n24.00.00 Авиационная и ракетно-космическая т...
3,https://openedu.ru/course/misis/EMQ/,Эмоциональный интеллект,\nКурс направлен на формирование универсальных...,\nСтруктура курса: Курс состоит из 10 недель и...,\n38.03.01 Экономика\n\n38.03.02 Менеджмент\n\...
4,https://openedu.ru/course/eltech/probability_t...,Теория вероятностей,\nКурс рассчитан на студентов бакалавриата и с...,\nВведение\nТема 1. Случайные события\nОсновны...,\n01.03.02 Прикладная математика и информатика...
...,...,...,...,...,...
498,https://openedu.ru/course/misis/EMQ/,Эмоциональный интеллект,\nКурс направлен на формирование универсальных...,\nСтруктура курса: Курс состоит из 10 недель и...,\n38.03.01 Экономика\n\n38.03.02 Менеджмент\n\...
499,https://openedu.ru/course/mipt/ELEC/,Электричество и магнетизм,\nСлушателям курса необходимо владеть знаниями...,\n\nЭлектрические заряды и электрическое поле....,\n03.00.00 Физика и астрономия\n\n04.00.00 Хим...
500,https://openedu.ru/course/mephi/mephi_res/,Радиационная экология,\n\n\n\n\nРО-1\n\n\nПонимание закономерностей ...,\nТема 1. Радиационная экология: история разви...,\n14.03.02 Ядерные физика и технологии\n\n14.0...
501,https://openedu.ru/course/miit/miit1/,Экономические основы транспортной деятельности,\n● Применение основ по методам реализации осн...,\nКурс состоит из двенадцати разделов:\nРаздел...,\n09.00.00 Информатика и вычислительная техник...
